In [3]:
%reload_ext autoreload
%autoreload 2

In [4]:
import numpy as np
from joblib import Parallel, delayed
from itertools import product
import os

import sys
sys.path.append(os.environ['GORDON_REPO_DIR'] + '/pipeline_scripts')
from utilities2014 import *

import matplotlib.pyplot as plt
%matplotlib inline

from tables import *

# import compute_pie

In [5]:
dm = DataManager(generate_hierarchy=False, stack='RS141', resol='x5', section=2)
dm._load_image()

/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/skimage/filter/__init__.py:6: skimage_deprecation: The `skimage.filter` module has been renamed to `skimage.filters`.  This placeholder module will be removed in v0.13.
  warn(skimage_deprecation('The `skimage.filter` module has been renamed '


In [6]:
textonmap = dm.load_pipeline_result('texMap', 'npy')
textonmap_viz = dm.load_pipeline_result('texMap', 'png')
n_texton = len(np.unique(textonmap)) - 1

mys, mxs = np.where(dm.mask)
mys = mys.astype(np.int16)
mxs = mxs.astype(np.int16)

In [7]:
n_theta = 12
theta_binwidth = 2*np.pi/n_theta

In [19]:
circle_indices_r = []

In [20]:
# if os.path.exists('/home/yuncong/csd395/H'):

#     with open_file('/home/yuncong/csd395/H', mode="r") as f:
#         H = f.root.data.read()

# else:

radius_list = [50,100,200]
n_radius = len(radius_list)
max_radius = radius_list[-1]

box_indices = np.dstack(np.meshgrid(np.arange(-max_radius, max_radius+1), 
                                    np.arange(-max_radius, max_radius+1))).astype(np.int16)
norms = np.linalg.norm(box_indices.astype(np.float), axis=-1)

pie_indices_r = []
for ri in range(n_radius):
#     if ri == 0:
#         circle_indices = box_indices[norms <= radius_list[0]]
#     else:
#         circle_indices = box_indices[(norms <= radius_list[ri]) & (norms > radius_list[ri-1])]

    circle_indices = box_indices[norms <= radius_list[ri]]
    circle_indices_r.append(circle_indices)
    
    
    angles = np.arctan2(circle_indices[:,0], circle_indices[:,1])
    angles[angles < 0] += 2*np.pi
    angular_bins = np.int0(angles/theta_binwidth)
    pie_indices = [circle_indices[angular_bins==k] for k in range(n_theta)]
    pie_indices_r.append(pie_indices)

In [ ]:
pie_mask = [[None for _ in range(n_theta)] for _ in radius_list]
for ri, r in enumerate(radius_list):
    for s in range(n_theta):
        pie_mask[ri][s] = np.zeros((2*r+1,2*r+1), np.bool).copy()
        pie_mask[ri][s][r+pie_indices_r[ri][s][:,0], r+pie_indices_r[ri][s][:,1]] = 1

In [8]:
# for ri, r in enumerate(radius_list):
#     for s in range(n_theta):
#         plt.figure()
#         plt.matshow(pie_mask[ri][s], cmap=plt.cm.gray_r)

In [9]:
from scipy.signal import convolve2d, fftconvolve

In [10]:
# t = time.time()
# # q = ndimage.convolve(textonmap_individual_channels[0], pie_mask[0][0])
# q = fftconvolve(textonmap_individual_channels[0], pie_mask[0][0], mode='same').astype(np.int)
# print time.time() - t

In [11]:
# t = time.time()
# q = compute_pie_histogram(1, 0, pie_mask[1][0])
# print time.time() - t

In [12]:
textonmap_individual_channels = [textonmap == c for c in range(n_texton)]

filters = Filters(complevel=9, complib='blosc')

def compute_pie_histogram(ri, s, pie_mask):
    h = np.zeros((dm.image_height, dm.image_width, n_texton), np.int)
    for c in range(n_texton):
        h[:,:,c] = fftconvolve(textonmap_individual_channels[c], pie_mask, mode='same').astype(np.int)
#         np.save('/home/yuncong/csd395/h_r%d_s%d.npy'%(ri,s), h)
        
    with open_file('/home/yuncong/csd395/h_r%d_s%d.hdf'%(ri,s), mode="w") as f:
        _ = f.create_carray('/', 'data', Int64Atom(), filters=filters, obj=h)
        
#     return h

In [13]:
from scipy import ndimage

for ri in range(n_radius):
    t = time.time()
    Parallel(n_jobs=16)(delayed(compute_pie_histogram)(ri, s, pie_mask[ri][s]) for s in range(n_theta))
    print time.time() - t

44.5730931759
47.9857242107
50.1922280788


In [ ]:
# with open_file('/home/yuncong/csd395/H_r%d.hdf'%ri, mode="w") as f:
#     for s in range(n_theta):
#         _ = f.create_carray('/', 'orientation_%d'%s, Int64Atom(), filters=filters, obj=H[s])

In [ ]:
# filters = Filters(complevel=9, complib='blosc')

# for ri in range(n_radius):

#     b = time.time()

#     H = Parallel(n_jobs=16)(delayed(compute_pie.compute_pie_histogram)(textonmap, pie_indices_r[ri][s], 
#                                                        mys, mxs, radius_list[ri], height, width, n_texton) 
#                             for s in range(n_theta))

#     print 1, time.time() - b
    
#     b = time.time()
    
#     with open_file('/home/yuncong/csd395/H_r%d.hdf'%ri, mode="w") as f:
#         for s in range(n_theta):
#             _ = f.create_carray('/', 'orientation_%d'%s, Int64Atom(), filters=filters, obj=H[s])

    
#     print 2, time.time() - b
    
#     del H

In [8]:
def chi2s(h1s, h2s):
    return np.nansum((h1s-h2s)**2/(h1s+h2s).astype(np.float), axis=1)

def compute_halfdisc_histogram_diff(H, start_bin, mys, mxs):
    
    n_theta, height, width = H.shape[:3]
    
    Gs = np.zeros((height, width), np.float)

    first_half_bins = np.arange(start_bin, start_bin+n_theta/2)%n_theta
    second_half_bins = np.arange(start_bin+n_theta/2, start_bin+n_theta)%n_theta

    H_halfdisk1 = np.sum(H[first_half_bins[:,None], mys, mxs], axis=0).astype(np.float)
    H_halfdisk2 = np.sum(H[second_half_bins[:,None], mys, mxs], axis=0).astype(np.float)
        
    H_halfdisk1 /= H_halfdisk1.sum(axis=-1)[:,None]
    H_halfdisk2 /= H_halfdisk2.sum(axis=-1)[:,None]
    
    Gs[mys, mxs] = chi2s(H_halfdisk1, H_halfdisk2)

    return Gs

In [11]:
# %time G = Parallel(n_jobs=16)(delayed(compute_halfdisc_histogram_diff)(s, mys, mxs) for s in range(n_theta/2))

# Gmax = np.zeros((dm.image_height, dm.image_width))
# H = np.zeros((n_theta, dm.image_height, dm.image_width, n_texton), np.int)

# for ri in range(n_radius):
    
#     t = time.time()

#     for s in range(n_theta):
#         with open_file('/home/yuncong/csd395/h_r%d_s%d.hdf'%(ri,s), mode="r") as f:
#             H[s] = f.get_node('/data').read()

#     #     G = np.empty((n_theta/2, height, width))
#     G = [compute_halfdisc_histogram_diff(H, s, mys, mxs) for s in range(n_theta/2)]
#     Gmax = np.maximum(Gmax, np.max(G, axis=0))
    
#     print time.time() - t
    
# Gmax = np.zeros((n_radius, dm.image_height, dm.image_width), np.float)
# for ri in range(n_radius):

ri = 0
for s in range(n_theta):
    with open_file('/home/yuncong/csd395/h_r%d_s%d.hdf'%(ri,s), mode="r") as f:
        H[s] = f.get_node('/data').read()
#     G = np.empty((n_theta/2, height, width))
G = [compute_halfdisc_histogram_diff(H, s, mys, mxs) for s in range(n_theta/2)]
Gmax = np.max(G, axis=0)



/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/IPython/kernel/__main__.py:16: RuntimeWarning: invalid value encountered in divide
/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/IPython/kernel/__main__.py:17: RuntimeWarning: invalid value encountered in divide
/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/IPython/kernel/__main__.py:2: RuntimeWarning: invalid value encountered in divide
  from IPython.kernel.zmq import kernelapp as app


In [12]:
Gmax_viz = plt.cm.jet(Gmax/np.nanmax(Gmax));
display(Gmax_viz)

/oasis/projects/nsf/csd395/yuncong/Brain/notebooks/tmp.jpg

In [13]:
np.save('/home/yuncong/csd395/Gmax.npy', Gmax)

In [ ]:
Gmax = np.load('/``home/yuncong/csd395/Gmax.npy')

In [ ]:
theta_ind = 0

In [ ]:
dy, dx = np.gradient(Gmax)

grad_norm = np.sqrt(dx**2+dy**2)
grad_unit_vec_y = np.nan_to_num(dy/grad_norm)
grad_unit_vec_x = np.nan_to_num(-dx/grad_norm)

gy = grad_unit_vec_y[mys, mxs]
gx = grad_unit_vec_x[mys, mxs]

a = np.arange(-20, 20)

neighborhood_y = np.outer(gy, a)
neighborhood_x = np.outer(gx, a)

global_neighborhood_y = (mys[:,None] + neighborhood_y).astype(np.int)
global_neighborhood_x = (mxs[:,None] + neighborhood_x).astype(np.int)

b = time.time()
global_neighborhood_y[global_neighborhood_y < 0] = 0
global_neighborhood_y[global_neighborhood_y >= height] = height-1
global_neighborhood_x[global_neighborhood_x < 0] = 0
global_neighborhood_x[global_neighborhood_x >= width] = width-1
print time.time()-b

# 60s, slower than above
# b = time.time()
# global_neighborhood_y = np.minimum(np.maximum(global_neighborhood_y.astype(np.int), 0), height)
# global_neighborhood_x = np.minimum(np.maximum(global_neighborhood_x.astype(np.int), 0), width)
# print time.time()-b

global_neighborhood_values = Gmax[global_neighborhood_y, global_neighborhood_x]
global_neighborhood_maximum = global_neighborhood_values.max(axis=1)

values = Gmax[mys, mxs]

is_local_maximum = global_neighborhood_maximum == values

mys_local_max = mys[is_local_maximum]
mxs_local_max = mxs[is_local_maximum]

G_nonmaxsup = np.zeros_like(Gmax)
G_nonmaxsup[mys_local_max, mxs_local_max] = Gmax[mys_local_max, mxs_local_max]
G_nonmaxsup[(grad_unit_vec_y==0)&(grad_unit_vec_x==0)] = 0

In [ ]:
np.save('/home/yuncong/csd395/G_nonmaxsup.npy', G_nonmaxsup)

In [ ]:
# plt.matshow(G_nonmaxsup);

# viz = plt.cm.jet(Gmax/Gmax.max())
# display(viz)

viz = plt.cm.jet(G_nonmaxsup/G_nonmaxsup.max());
display(viz)

In [ ]:
G_nonmaxsup = np.load('/home/yuncong/csd395/G_nonmaxsup.npy')

Compute intervening contour cue

In [27]:
r = 5
rho = .5

In [28]:
conns_ij_y = []
conns_ij_x = []
circle_j = []
for y, x in product(range(-r, r+1), range(-r, r+1)):
    d = np.sqrt(y**2+x**2)
    if d < r and not (y==0 and x==0):
        pts_conn_ij_y = np.linspace(0,y,d).astype(np.int)
        pts_conn_ij_x = np.linspace(0,x,d).astype(np.int)
        circle_j.append((y,x))
        conns_ij_y.append(pts_conn_ij_y)
        conns_ij_x.append(pts_conn_ij_x)

circle_j = np.asarray(circle_j)

In [ ]:
import compute_pie

b = time.time()

A = Parallel(n_jobs=16)(delayed(compute_pie.compute_connection_weight)(G_nonmaxsup, conns_ij_y, conns_ij_x,
                                                                       mys[s], mxs[s], height, width)
                        for s in np.array_split(range(len(mys)), 16))

print time.time()-b

A = np.vstack(A)
W = np.exp(-A/rho)

Alternative: use iPython Parallel

In [ ]:
from IPython.parallel import Client
cli = Client(profile='parallel')
dview = cli[:]

In [ ]:
dview.execute("import sys; sys.path.append('/home/yuncong/csd395/Brain/notebooks')")
dview.execute('import compute_pie')

mydict=dict(G_nonmaxsup=G_nonmaxsup, conns_ij_y=conns_ij_y,
             conns_ij_x=conns_ij_x, height=height, width=width)
dview.push(mydict)

bar = lambda a,b: compute_pie.compute_connection_weight(G_nonmaxsup=G_nonmaxsup, conns_ij_y=conns_ij_y,
             conns_ij_x=conns_ij_x, height=height, width=width, mys=a, mxs=b)

b = time.time()
A = dview.map_sync(bar, np.array_split(mys, 192), np.array_split(mxs, 192))
print time.time() - b

A = np.vstack(A)
W = np.exp(-A/rho)
del A

In [24]:
with open_file('/home/yuncong/csd395/A.hdf', mode="r") as f:
    A = f.get_node('/data').read()

In [29]:
pis = np.c_[mys, mxs]
pjs = pis[:,None] + circle_j
lookup = dict([(tuple(pi), i) for i, pi in enumerate(pis)])
nz_is, nz_offsets = np.where(A)
print len(nz_is)

def lookup_point_indices(pts):
    return [lookup[tuple(p)] for p in pts]

b = time.time()

Q = Parallel(n_jobs=16)(delayed(lookup_point_indices)(pts) for pts in np.array_split(pjs[nz_is, nz_offsets], 16))

from itertools import chain
js = np.fromiter(list(chain(*Q)), dtype=np.int)

print time.time() - b

In [34]:
from scipy.sparse import coo_matrix, csr_matrix, dia_matrix, diags

In [65]:
W = np.exp(-A/rho) # breaks sparsity

In [88]:
n = len(mys)
M = coo_matrix((A[nz_is, nz_offsets], (nz_is, js)), shape=(n,n))
m = M.tocsr()

In [144]:
partial_m = m[:20000,:20000].todense()
W = np.exp(-partial_m/rho)
D = np.diag(W.sum(axis=1).A1)

In [124]:
D = diags([W.sum(axis=1).A1], [0])
d = D.tocsr()

In [5]:
def save_sparse_csr(filename,array):
    np.savez(filename,data = array.data ,indices=array.indices,
             indptr =array.indptr, shape=array.shape )

def load_sparse_csr(filename):
    loader = np.load(filename)
    return csr_matrix((  loader['data'], loader['indices'], loader['indptr']),
                         shape = loader['shape'])

In [6]:
L = load_sparse_csr("/home/yuncong/csd395/L.npz")
d = load_sparse_csr("/home/yuncong/csd395/d.npz")

In [39]:
import scipy.sparse.linalg

In [145]:
vals, vecs = scipy.sparse.linalg.eigsh(D-W, k=4, M=D)

In [147]:
I = np.zeros(image.shape[:2])
I[mys[:20000], mxs[:20000]] = vecs[:,0]
plt.matshow(I)

In [ ]:
#     dy, dx = np.gradient(G[theta_ind])
#     dt = dx*np.cos(diff_orientation) + dy*np.sin(diff_orientation)

Gmax_viz = plt.cm.jet(G[theta_ind]/np.nanmax(G[theta_ind]));
display(Gmax_viz)

In [ ]:
display(image)

In [ ]:
display(textonmap_viz)

In [ ]:
from scipy.ndimage.filters import gaussian_filter

g = np.empty_like(G)
eps = .001
sigma = 10

for theta_ind in range(n_theta/2):
    print theta_ind
    
#     diff_orientation = thetas[theta_ind] + np.pi/2

#     dy, dx = np.gradient(G[theta_ind])
#     dt = dx*np.cos(diff_orientation) + dy*np.sin(diff_orientation)
    
#     dt_dy, dt_dx = np.gradient(dt)
#     dt_dt = dt_dx*np.cos(diff_orientation) + dt_dy*np.sin(diff_orientation)

#     d_to_peak = -(np.abs(dt)+eps)/dt_dt
#     g = G[theta_ind]/(d_to_peak)
    
    dt = gaussian_filter(G[theta_ind], sigma, order=1)
    dt_dt = gaussian_filter(G[theta_ind], sigma, order=2)
    g[theta_ind] = G[theta_ind]*(-dt_dt/(np.abs(dt)+eps))

In [ ]:
gmax = np.max(np.abs(g), axis=0)
gmax = gmax/gmax.max()
gmax_viz = plt.cm.jet(gmax*2);
display(gmax_viz)

In [ ]:
vmax = np.nanmax(G)
thetas = np.linspace(0, np.pi, n_theta/2+1)

n_cols = 2
n_rows = n_theta/2/n_cols
fig, axes = plt.subplots(n_rows, n_cols, figsize=(20,20));
for s in range(n_theta/2):
    ax = axes[s/n_cols, s%n_cols]
    ax.matshow(G[s], vmin=0, vmax=vmax);
    ax.set_title('theta = %.2f'%np.rad2deg(thetas[s]))
    ax.set_axis_off();